# OME-Zarr Image Exploration

In this notebook we will show how to use the 'OmeZarr Container' class to explore and manage an OME-NGFF image.

For this example we will use a small example image that can be downloaded from the following link: [example ome-zarr](https://zenodo.org/records/13305156)

## OmeZarr Container

The `OmeZarr Container` provides a high-level interface to read, write and manipulate NGFF images.
A `OmeZarr Container` can be created from a storelike object (e.g. a path to a directory, or a url) or from a `zarr.Group` object. 

In [ ]:
from pathlib import Path

from ngio import open_omezarr_container
from ngio.utils import download_ome_zarr_dataset

# Download the dataset
download_dir = Path(".").absolute().parent.parent / "data"
hcs_path = download_ome_zarr_dataset("CardiomyocyteSmallMip", download_dir=download_dir)
image_path = hcs_path / "B" / "03" / "0"

# Open the ome-zarr container
omezarr_container = open_omezarr_container(image_path)


The `omezarr_container` object provides a high-level interface to read, write and manipulate OME-Zarr images.

Print the image will show some overview information like:
* The path to the image
* The multiscale pyramid paths
* The labels contained in the image
* The tables contained in the imag

In [ ]:
print(omezarr_container)

From the `OmeZarr Container` object we can easily access access the image data (at any resolution level), the labels and the tables.

Get a single `level` of the image pyramid as `Image` (to know more about the `Image` class, please refer to the [Image notebook](./image.ipynb)
The `Image` object is the main object to interact with the image. 
It contains methods to interact with the image data and metadata.

In [ ]:
from ngio import PixelSize

# 1. Get image from highest resolution (default)
image = omezarr_container.get_image()
print(image)

# 2. Get image from a specific level using the path keyword
image = omezarr_container.get_image(path="1")
print(image)

# 3. Get image from a specific pixel size using the pixel_size keyword
# image = omezarr_container.get_image(
#    pixel_size=PixelSize(x=0.65, y=0.65, z=1), strict=True
# )
print(image.pixel_size == PixelSize(x=0.325, y=0.325, z=1))

The `Image` object provides a high-level interface to read and write image data at a specific resolution level.

In [ ]:
print("Shape", image.shape)
print("PixelSize", image.pixel_size)
print("Dimensions", image.dimensions)
print("Channel Names", image.channel_labels)

In [ ]:
# Get data as a numpy array or a dask array
data = image.get_array(c=0, mode="numpy")
print(data)

dask_data = image.get_array(c=0, mode="dask")
dask_data

`ngio` design is to always provide the data in a canonical axis order (`t`, `c`, `z`, `y`, `x`) no matter what is the order on disk.  
The `Image` object provides methods to access the data in this order.  


## Labels

The `NgffImage` can also be used to load labels from a `OME-NGFF` file and behave similarly to the `Image` object.

In [ ]:
print("List of Labels: ", omezarr_container.list_labels())

label_nuclei = omezarr_container.get_label("nuclei", path="0")
print(label_nuclei)

## Tables

The `NgffImage` can also be used to load tables from a `OME-NGFF` file.

`ngio` supports three types of tables:
 - `features table` A simple table to store features associated with a label.
 - `roi table` A table to store regions of interest.
 - `masking roi tables` A table to store single objects bounding boxes associated with a label.

In [ ]:
print("List of Tables: ", omezarr_container.list_tables())

In [ ]:
# Loading a table
feature_table = omezarr_container.get_table("regionprops_DAPI")
feature_table.dataframe

In [ ]:
# Loading a roi table
roi_table = omezarr_container.get_table("FOV_ROI_table")

print(f"{roi_table.get('FOV_1')=}")

Rois can be used to index image and label data.

In [ ]:
import matplotlib.pyplot as plt

# Plotting a single ROI
roi = roi_table.get("FOV_1")
roi_data = image.get_roi(roi, c=0, mode="numpy")
plt.title("ROI: FOV_1")
plt.imshow(roi_data[0, 0], cmap="gray")
plt.axis("off")
plt.show()

## Derive a new NgffImage

When processing an image, it is often useful to derive a new image from the original image.
The `NgffImage` class provides a method to derive a new image from the original image.
When deriving a new image, a new `NgffImage` object is created with the same metadata as the original image. Optionally the 
user can specify different metadata for the new image(.e.g. different channels names).

In [ ]:
new_omezarr_image = omezarr_container.derive_image("data/new_ome.zarr", overwrite=True)
print(new_omezarr_image)

# Create an OmeZarr From a Numpy Array


In [ ]:
import numpy as np

from ngio import create_omezarr_from_array

x = np.random.randint(0, 255, (16, 128, 128), dtype=np.uint8)

new_omezarr_image = create_omezarr_from_array(
    store="random_ome.zarr", array=x, xy_pixelsize=0.65, z_spacing=1.0
)
print(new_omezarr_image)
print(new_omezarr_image.get_image())

## Steam an OmeZarr over HTTP

The `OmeZarr` class can also be used to stream an image over HTTP. This is useful when the image is stored on a remote server and you want to access it without downloading the entire image. All features of the `OmeZarr` class are available when streaming an image over HTTP (besides anything that requires writing to the image).

In [ ]:
import fsspec
import fsspec.implementations.http

url = (
    "https://raw.githubusercontent.com/"
    "fractal-analytics-platform/fractal-ome-zarr-examples/"
    "refs/heads/main/v04/"
    "20200812-CardiomyocyteDifferentiation14-Cycle1_B_03_mip.zarr/"
)

fs = fsspec.implementations.http.HTTPFileSystem(client_kwargs={})
store = fs.get_mapper(url)
omezarr = open_omezarr_container(store)
omezarr

# Streaming an OmeZarr from a Fractal Server

Example:

```python
from ngio.utils import fractal_fsspec_store

store = fractal_fsspec_store(url="https://fracral_url...", fractal_token="**your_secret_token**")
omezarr = open_omezarr_container(store)
omezarr
```